In [1]:
import threading

In [18]:
def counter(n):
    print("start")
    th = threading.current_thread()
    print(th, th.name, th.is_alive(), th.daemon, th.ident, th.native_id)
    while n > 0:
        n -= 1

        
th = threading.Thread(
    target=counter,
    args=(100000000,),
    name="counter_example"
)
th.start()
print("before")
th.join()
print("after")


# counter(*args, **kwargs)

startbefore

<Thread(counter_example, started 123145611214848)> counter_example True False 123145611214848 13315028
after


In [11]:
threading.active_count()

8

In [13]:
threading.current_thread(), threading.current_thread().name

(<_MainThread(MainThread, started 4560991744)>, 'MainThread')

In [14]:
threading.enumerate()

[<_MainThread(MainThread, started 4560991744)>,
 <Thread(IOPub, started daemon 123145492078592)>,
 <Heartbeat(Heartbeat, started daemon 123145508868096)>,
 <Thread(Thread-3 (_watch_pipe_fd), started daemon 123145526730752)>,
 <Thread(Thread-4 (_watch_pipe_fd), started daemon 123145543520256)>,
 <ControlThread(Control, started daemon 123145560309760)>,
 <HistorySavingThread(IPythonHistorySavingThread, started 123145577099264)>,
 <ParentPollerUnix(Thread-2, started daemon 123145594425344)>]

In [49]:
N = 100000
ITERS = 1000
# counter_calls = 0

def _counter(n):
#     global counter_calls
#     counter_calls += 1

    while n >= 0:
        n -= 1

        
def countdown(iters):
    for i in range(iters):        
        _counter(N)


In [44]:
%%time

countdown(ITERS)
print(counter_calls)

1000
CPU times: user 4.46 s, sys: 55.8 ms, total: 4.51 s
Wall time: 4.97 s


In [39]:
%%time

th = threading.Thread(
    target=countdown,
    args=(ITERS,),
    name="counter_example"
)
th.start()
th.join()

CPU times: user 4.52 s, sys: 49.5 ms, total: 4.57 s
Wall time: 5.03 s


In [47]:
%%time

N_THREADS = 100


threads = [
    threading.Thread(
        target=countdown,
        args=(ITERS // N_THREADS,),
    )
    for _ in range(N_THREADS)
]

for th in threads:
    th.start()

for th in threads:
    th.join()
    
    
print(counter_calls)

1000
CPU times: user 4.4 s, sys: 97.9 ms, total: 4.5 s
Wall time: 4.67 s


In [48]:
#GIL

In [50]:
from urllib.request import urlopen

URL = "https://ru.wikipedia.org/wiki/Python"
N = 100


def fetch_url(url):
    resp = urlopen(url)
    return resp


def fetch_batch_urls(url, times):
    for _ in range(times):
        resp = fetch_url(url)

In [51]:
%%time

fetch_batch_urls(URL, N)

CPU times: user 957 ms, sys: 109 ms, total: 1.07 s
Wall time: 16.6 s


In [53]:
%%time

N_THREADS = 1


threads = [
    threading.Thread(
        target=fetch_batch_urls,
        args=(URL, N // N_THREADS,),
    )
    for _ in range(N_THREADS)
]

for th in threads:
    th.start()

for th in threads:
    th.join()

CPU times: user 962 ms, sys: 104 ms, total: 1.07 s
Wall time: 16.3 s


In [55]:
%%time

N_THREADS = 10


threads = [
    threading.Thread(
        target=fetch_batch_urls,
        args=(URL, N // N_THREADS,),
    )
    for _ in range(N_THREADS)
]

for th in threads:
    th.start()

for th in threads:
    th.join()

CPU times: user 952 ms, sys: 105 ms, total: 1.06 s
Wall time: 2.18 s


In [60]:
%%time

import time

global_count = [0]


def danger(iters):
    for _ in range(iters):
        c = global_count[0]
        c += 1
        time.sleep(0.0000000001)
        global_count[0] = c
        
        #global_count[0] += 1


threads = [
    threading.Thread(
        target=danger,
        args=(10,),
    )
    for _ in range(10000)
]

for th in threads:
    th.start()

for th in threads:
    th.join()
    
    
print(global_count)

[18679]
CPU times: user 1.08 s, sys: 1.51 s, total: 2.59 s
Wall time: 1.71 s


In [63]:
%%time

import time

global_count = [0]
lock = threading.Lock()


def danger_atomic(iters, lock):
    for _ in range(iters):
        lock.acquire()
        
        c = global_count[0]
        c += 1
        time.sleep(0.0000000001)
        global_count[0] = c
        
        lock.release()


threads = [
    threading.Thread(
        target=danger_atomic,
        args=(100, lock),
    )
    for _ in range(1000)
]

for th in threads:
    th.start()

for th in threads:
    th.join()
    
    
print(global_count)

[100000]
CPU times: user 458 ms, sys: 2.01 s, total: 2.47 s
Wall time: 1.79 s


In [69]:
%%time

from urllib.request import urlopen

URL = "https://ru.wikipedia.org/wiki/Python"
N = 100
N_THREADS = 10

sem = threading.Semaphore(2)


def fetch_url(url, sem):
    with sem:
        resp = urlopen(url)
    return resp


def fetch_batch_urls(url, times, sem):
    for _ in range(times):
        resp = fetch_url(url, sem)
        

threads = [
    threading.Thread(
        target=fetch_batch_urls,
        args=(URL, N // N_THREADS, sem),
    )
    for _ in range(N_THREADS)
]

for th in threads:
    th.start()

for th in threads:
    th.join()

CPU times: user 998 ms, sys: 108 ms, total: 1.11 s
Wall time: 9.01 s


In [79]:
%%time

from urllib.request import urlopen
import queue


URL = "https://ru.wikipedia.org/wiki/Python"
N = 100
N_THREADS = 10


sem = threading.Semaphore(10)
que = queue.Queue(20)


def fetch_url(sem, que):
    while True:
        try:
            url = que.get(timeout=1)
            if url is None:
                break
        except Exception:
            continue

        with sem:
            resp = urlopen(url)
            print("status", resp.code)
        

threads = [
    threading.Thread(
        target=fetch_url,
        args=(sem, que),
    )
    for _ in range(N_THREADS)
]

for th in threads:
    th.start()


for _ in range(N):
    que.put(URL)

    
for _ in range(N_THREADS):
    que.put(None)

    
for th in threads:
    th.join()

status 200
status 200
status 200
status 200
status 200
status 200
status 200
status 200
status 200
status 200
status 200
status 200
statusstatus 200
status 200
status 200
 200
status 200
status 200
status 200
status 200
status 200
status 200
status 200
status 200
status 200
status 200
status 200
status 200
status 200
status 200
status 200
status 200
status 200
status 200
status 200
status 200
status 200
status 200
status 200
status 200
status 200
status 200
status 200
status 200
status 200
status 200
status 200
status 200
status 200
status 200
status 200
status 200
status 200
status 200
status 200
status 200
status 200
status 200
status 200
status 200
status 200
status 200
status 200
status 200
statusstatus 200
 200
status 200
status 200
status 200
status 200
status 200
status 200
status 200
status 200
status 200
status 200
status 200
status 200
status 200
status 200
status 200
status 200
status 200
status 200
status 200
status 200
status 200
status 200
status 200
status 200
status 200

In [82]:
import multiprocessing


class A:
    x = 42
    

def multy_func(dct):
    dct["data"] = A()
    dct["qwerty"] = 99


with multiprocessing.Manager() as manager:
    dct = manager.dict()
    
    print("before", dct)
    
    p = multiprocessing.Process(target=multy_func, args=(dct,))
    p.start()
    p.join()
    
    print("after", dct)

before {}
after {}


Traceback (most recent call last):
  File "<string>", line 1, in <module>
  File "/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/multiprocessing/spawn.py", line 116, in spawn_main
    exitcode = _main(fd, parent_sentinel)
  File "/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/multiprocessing/spawn.py", line 126, in _main
    self = reduction.pickle.load(from_parent)
AttributeError: Can't get attribute 'multy_func' on <module '__main__' (built-in)>
